In [115]:
import nltk
from nltk.tokenize import sent_tokenize
import os
from langchain.llms import HuggingFaceHub
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from huggingface_hub import InferenceClient
import torch
from transformers import pipeline
import requests
import runpod
from text_generation import Client
import json
import re
import tiktoken
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
api_key = '0QGPOO4GHY3230X2I51YWXTXP8E3UDG0AVNOCO9O'

In [125]:
def clean_whitespace(text):
        text = re.sub(r'\n{2,}', '\n', text)
        text = re.sub(r' {2,}', '\n', text)
        text = text.strip()
        return text

In [139]:
def load_data(data):
    with open(data, 'r') as f:
        story = f.read()
    
    clean_story = clean_whitespace(story)
    
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = encoding.encode(clean_story)
    print(len(num_tokens))

    segments = []
    current_segment = []
    
    # will loop untill list has 4050 tokens
    for token in num_tokens:
        current_segment.append(token)
        
        # if >= 4069 will add this batch to new list
        if len(current_segment) >= 4069:
            segment_text = encoding.decode(current_segment)
            segments.append(segment_text + '\n\n')
            current_segment = []
            
    if current_segment:
        segment_text = encoding.decode(current_segment)
        segments.append(segment_text)
     
    return segments

load_data("D:\coding\llms\sci_storys\story1.txt")

2066


['Thornton Gombar lifted his gaze to the sky as a hovercraft zoomed over his home. Based on the rosy pink tints that adorned its exterior, Thornton surmised that the vehicle was a pleasure craft, likely filled with a small group of men and women engaging in gross debauchery as the self-driving vehicle propelled itself across the sky. The craft jerked and jolted in an arrhythmic manner as it flew to the heart of Homasoro City, where even more such activities awaited them. \nThornton shook his head and turned his attention again to the ground. \nBefore his home lay a small plot of brownish soil. He had spent the entire day clearing it of the gnarled weeds that had accumulated during decades of neglect. Only one remained. \nThornton got on his knees and, wrapping his fingers around the base, began to pull. The weed didn\'t provide much resistanceâ€”immediately, a resounding \'snap\' echoed through the yard as the stem broke apart. \nThornton tossed the weed aside and stared forlornly at t

In [59]:
def gen_prompt(input_text, api_key):
    url = "https://api.runpod.ai/v2/llama2-13b-chat/runsync"
    prompt = f"""
    Give a short and concise output.
    Given the following story segment: '{input_text}', generate a suitable prompt that a user would ask for if they wanted a cool sci-fi story.
    DO NOT give me any additional words other than what is asked.
    """

    payload = { "input": {
        "prompt": prompt,
        "sampling_params": {
            "max_tokens": 1000,
            "n": 1,
            "best_of": None,
            "presence_penalty": 0,
            "frequency_penalty": 0,
            "temperature": 0.7,
            "top_p": 1,
            "top_k": -1,
            "use_beam_search": False,
            "stop": ["None"],
            "ignore_eos": False,
            "logprobs": None
        }
    } }
    headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": api_key
    }
    
    response = requests.post(url, json=payload, headers=headers)

    response_data = response.json()

    job_id = response_data.get('id')
    return job_id  
    
input_text = 'It was approaching 2am when Lexi made the discovery that would fracture the world. She skulled the rest of her black coffee and sat the paper cup down on the smooth, white table. She was at the university working on her PhD thesis. The research involved creating controlled, miniature black holes in a Vacuity Machine and then testing its potential for hyper-space travel.'     
job_id = gen_prompt(input_text, api_key)

def poll_for_result(request_id, api_key, interval=5):
    status_url = f"https://api.runpod.ai/v2/llama2-13b-chat/status/{request_id}"  
    headers = {
        "accept": "application/json",
        "authorization": api_key
    }

    while True:
        response = requests.get(status_url, headers=headers)
        result = response.json()

        if result['status'] == 'COMPLETED':
            output = result.get('output')
            return output  
        elif result['status'] in ['FAILED', 'ERROR']:
            print("Error or Failed Status:", result)
            return result

        time.sleep(interval)  # Wait before polling again



if job_id:
    output = poll_for_result(job_id, api_key)
    print("Final Output:", output)

Final Output: {'input_tokens': 161, 'output_tokens': 82, 'text': ["\n    Here's a prompt for a cool sci-fi story based on the given story segment:\n    'What if Lexi's discovery led to a catastrophic event that threatened the fabric of reality? Could she use her knowledge of the Vacuity Machine to prevent the impending doom, or would the power of the black holes prove too great for her to control?'"]}


In [102]:
extracted_text = output['text'][0] if output['text'] else None
segments = extracted_text.split('\n')
extracted_prompt = segments[2].strip() if len(segments) > 1 else None    
extracted_prompt = extracted_prompt.strip("'")


"What if Lexi's discovery led to a catastrophic event that threatened the fabric of reality? Could she use her knowledge of the Vacuity Machine to prevent the impending doom, or would the power of the black holes prove too great for her to control?"

In [101]:
json_obj = {
  "messages": [
    {"role": "system", "content": "You are the greatest sci-fi story author in the universe."},
    {"role": "user", "content": extracted_prompt},
    {"role": "assistant", "content": input_text}
  ]
}
json_obj

{'messages': [{'role': 'system',
   'content': 'You are the greatest sci-fi story teller the world has seen.'},
  {'role': 'user',
   'content': "What if Lexi's discovery led to a catastrophic event that threatened the fabric of reality? Could she use her knowledge of the Vacuity Machine to prevent the impending doom, or would the power of the black holes prove too great for her to control?"},
  {'role': 'assistant',
   'content': 'It was approaching 2am when Lexi made the discovery that would fracture the world. She skulled the rest of her black coffee and sat the paper cup down on the smooth, white table. She was at the university working on her PhD thesis. The research involved creating controlled, miniature black holes in a Vacuity Machine and then testing its potential for hyper-space travel.'}]}

In [33]:
def create_prompt(input_text):
    template = f"""You are the greatest sci-fi story teller in the universe. Take an input: {input_text}

Provide a prompt for this input. Of what exactly is happening in the story."""
    return template
